# Nettoyage de données

In [2]:
import pandas as pd
import numpy as np
import json
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

In [21]:
!curl -XDELETE http://localhost:9200/movies/

{"acknowledged":true}
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    21  100    21    0     0    122      0 --:--:-- --:--:-- --:--:--   122


In [22]:
LOCAL = True
es_client = Elasticsearch(hosts=["localhost" if LOCAL else "elastic"])

with open('dataFin.json') as json_data:
    v = json.load(json_data)

#print(v)

def generate_data_exemple(documents):
    for docu in documents:
        yield {
            "_index": "movies",
            "_type": "movie",
            "_source": {k: v if v else None for k, v in docu.items()},
        }

bulk(es_client, generate_data_exemple(v))

(30, [])

In [23]:
!curl http://localhost:9200/_cat/indices?v

health status index  uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   movies 7gNmflYSQx6O_yIweo4wNw   1   1         30            0     14.2kb         14.2kb
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   204  100   204    0     0   7285      0 --:--:-- --:--:-- --:--:--  7555


In [5]:
with open('dataFin.json') as json_data:
   v = json.load(json_data)

#print(v)

In [25]:
QUERY = {
    "sort" : [
        { "Evaluation spectateur" : {"order" : "desc"}},
    ],
    "query" : {
        "match_all": {}
    }
}

result = es_client.search(index="movies", body=QUERY)
{elt['_source']['titre']:elt['_source']['Evaluation spectateur']  for elt in result["hits"]["hits"]}

{'La vie est belle': 8.2,
 'Le Noël de Mickey': 7.5,
 "Maman, j'ai raté l'avion !": 6.3,
 'Christmas Story': 6.2,
 'Le Drôle de Noël de Scrooge': 6.2,
 'Le Pôle Express': 6.0,
 'Super Noël': 5.7,
 "Maman, j'ai encore raté l'avion...": 5.7,
 'Le Sapin a les boules': 5.7,
 'This Christmas': 5.6}